In [149]:
import pandas as pd
from scipy import stats
import numpy as np

In [150]:
df = pd.read_csv('/Users/yiwenwang/Downloads/movies_metadata.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [151]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [152]:
df.drop_duplicates(subset=['title'], inplace=True)
# 以title去重, subset里是列表可以放入多列数据

In [153]:
df.drop(df[df['adult'] == 'TRUE'].index, inplace=True)
# 去除adult为true

In [154]:
df.drop(df[df['vote_average'] == 0].index, inplace=True)
# 去除revenue为0

In [156]:
df['release_date'] = df['release_date'].apply(lambda x: str(x).split('-')[0])

In [157]:
df['genres'] = df['genres'].apply(lambda x: eval(x)).apply(lambda x:','.join([i['name'] for i in x]))
# 将genres中的name值提取出来

In [158]:
df_c = df['genres'].str.split(',', expand=True)
# 将genres的值拆分成列

In [159]:
df_c = df_c.add_prefix('genre_')

In [160]:
df = pd.concat([df, df_c], axis=1)

In [161]:
df =df = df.drop('homepage', 1)
df =df = df.drop('overview', 1)
df =df = df.drop('poster_path', 1)

In [162]:
df.shape

(39472, 29)

In [163]:
df.head()

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,...,vote_average,vote_count,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"Animation,Comedy,Family",862,tt0114709,en,Toy Story,21.9469,"[{'name': 'Pixar Animation Studios', 'id': 3}]",...,7.7,5415.0,Animation,Comedy,Family,None,None,None,None,None
1,False,NaN,65000000,"Adventure,Fantasy,Family",8844,tt0113497,en,Jumanji,17.0155,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",...,6.9,2413.0,Adventure,Fantasy,Family,None,None,None,None,None
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"Romance,Comedy",15602,tt0113228,en,Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",...,6.5,92.0,Romance,Comedy,None,None,None,None,None,None
3,False,NaN,16000000,"Comedy,Drama,Romance",31357,tt0114885,en,Waiting to Exhale,3.85949,[{'name': 'Twentieth Century Fox Film Corporat...,...,6.1,34.0,Comedy,Drama,Romance,None,None,None,None,None
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,11862,tt0113041,en,Father of the Bride Part II,8.38752,"[{'name': 'Sandollar Productions', 'id': 5842}...",...,5.7,173.0,Comedy,None,None,None,None,None,None,None
